In [1]:
import pandas as pd
import numpy as np
import funcs, gc, sqlite3

In [ ]:
# bos ou rat
tipo = 'bos'

In [3]:
unclassif = funcs.get_unclassified_registers(tipo=tipo)
unclassif

alterou


,RAT.NUM_ATIVIDADE,MUNICIPIO,LOGRADOURO,DES_ENDERECO,COMPLEMENTO_ENDERECO,NOME_BAIRRO,LOGRADOURO2,DES_ENDERECO2,SETOR,CIA
0,2020-014269725-001,DIVINOPOLIS,PERNAMBUCO,PERNAMBUCO,,,,,other,other


In [6]:
classificadores = funcs.get_df_classif()
unclassif.loc[:,'SETOR'] = unclassif.apply(lambda row: funcs.classifica_setor(row, classificadores), axis=1)
funcs.classifica_cia(unclassif)

unclassif

,RAT.NUM_ATIVIDADE,MUNICIPIO,LOGRADOURO,DES_ENDERECO,COMPLEMENTO_ENDERECO,NOME_BAIRRO,LOGRADOURO2,DES_ENDERECO2,SETOR,CIA
0,2020-014269725-001,DIVINOPOLIS,PERNAMBUCO,PERNAMBUCO,,,,,PLANALTO,139 CIA


In [7]:
new_classif_dict = [
    {
        'nrat': unclass_list[0],
        'setor': unclass_list[-2],
        'cia': unclass_list[-1]
    }
    for unclass_list in unclassif.values
]

new_classif_dict

[{'nrat': '2020-014269725-001', 'setor': 'PLANALTO', 'cia': '139 CIA'}]

In [8]:
query_update = '''
UPDATE tbl_{}
    SET "SETOR" = ?,
    "CIA" == ?
WHERE
    "RAT.NUM_ATIVIDADE" = ?
'''.format(tipo)

with sqlite3.connect('gdo.db') as conn:
    cursor = conn.cursor()
    for dict_class in new_classif_dict:
        cursor.execute(query_update, (dict_class['setor'], dict_class['cia'], dict_class['nrat']))
        conn.commit()
conn.close()